In [ ]:
import random
import statistics
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
from scipy.stats import linregress
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.constants import k as k_B  # Boltzmann constant in J/K
from scipy.constants import G, proton_mass
from functools import partial
# from lab_functions_1 import Ax, vcgrab, Tc, Lambdacalc, dLdTfunc, dVcdrfunc
import lab_functions_1 as lf
import WiersmaCopy as Cool
G = 6.6743*10**-8
mp_g = proton_mass*1e3
k_Bcgs = k_B*1e7
mu = 0.6
gamma = 5/3
ktc = 3.0857e21
etkv = 6.2415*10**8

In [ ]:
# Create an instance of the cooling object
Z2Zsun = 0.3  # solar metallicity
z = 0.6       # redshift
cooling = Cool.Wiersma_Cooling(Z2Zsun=Z2Zsun, z=z)

# Generate random test values for T and nH
n_samples = 5
T_samples = 10**np.random.uniform(4.2, 8.16, size=n_samples)      # T from 1e4 to 1e8 K
nH_samples = 10**np.random.uniform(-3.7, -0.52, size=n_samples)    # nH from 1e-6 to 1e2 cm^-3

# Test LAMBDA
print("T [K]       nH [cm^-3]     Lambda [erg cm^3 / s]")
for T, nH in zip(T_samples, nH_samples):
    Lambda_val = cooling.LAMBDA(T, nH)
    print(f"{T: .2e}   {nH: .2e}     {Lambda_val: .2e}")

In [ ]:
left = np.array([9.3, 9.5, 9.7, 9.9, 10.1, 10.3, 10.5, 10.7, 10.9, 11.1, 11.3, 11.5, 11.7, 11.9, 12.1, 12.3, 12.5, 12.7, 12.9, 
    13.1, 13.3, 13.5, 13.7, 13.9, 14.1, 14.3, 14.5, 14.7])

right = np.array([-4.560623, -4.336346, -4.1121, -3.887918, -3.663862, -3.440012, -3.216311, -2.991903, -2.763624, -2.524845, 
    -2.269454, -2.005525, -1.769973, -1.619732, -1.591587, -1.669076, -1.79781, -1.929445, -2.045038, -2.146987, -2.242711, 
    -2.336809, -2.430866, -2.525143, -2.61958, -2.714102, -2.808667, -2.903251])

f = interp1d(left, right, kind='cubic', fill_value='extrapolate')
print(f(15.2))

plt.plot(left, right, marker='o', linestyle='-', color='b')
plt.xlabel('log10(Halo Mass)')
plt.ylabel('log10(Stellar Mass)')
plt.title('Stellar Mass vs Halo Mass at z ~ 0.59')
plt.grid(True)
plt.show()

In [ ]:
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
z = 0.597
rho_m = cosmo.Om(z) * cosmo.critical_density(z)
rho_m = rho_m.value
x = cosmo.Om(z) - 1
delc = 18 * np.pi**2 + 82 * x - 39 * x**2
delm = delc / cosmo.Om(z)
Rvir = (3 * 2e15 * 1.988e33 / (4 * np.pi * delm * rho_m))**(1/3)
R500 = (3 * 2e15 * 1.988e33 / (4 * np.pi * 500 * cosmo.critical_density(z).value))**(1/3)
print(f"Rvir = {Rvir/ktc} rs = {Rvir/(5*ktc)}")
print(R500/ktc)

In [ ]:
print(mpl.colors.get_named_colors_mapping())

In [ ]:
y = np.log10(np.array([3324.598, 1522.680, 1553.475, 1297.309, 725.888, 594.173, 374.898, 346.043, 214.010, 193.623, 155.348, 137.762, 127.159, 112.765,
    96.075, 85.199, 78.642, 69.739, 57.085, 51.647, 39.811, 35.304, 33.246, 30.079, 16.830, 14.925, 14.055, 12.716, 9.607, 8.185,
    4.228, 3.008, 2.563, 1.975]))

y_pairs = y.reshape(-1, 2)

# Compute symmetric error for each pair
errors = np.abs(y_pairs[:, 0] - y_pairs[:, 1]) / 2
stdvi = errors**2

print(stdvi)

In [ ]:
T_grid = np.logspace(4.16, 10, 10000)
L_grid = np.log10(np.array([lf.Lambdacalc(T, 1000*ktc, "a", 1) for T in T_grid]))
L_grid2 = np.log10(np.array([lf.Lambdacalc(T, 1000*ktc, 0.3, 1) for T in T_grid]))

plt.scatter(T_grid, L_grid, s=1)
plt.scatter(T_grid, L_grid2, s=1)
plt.xlabel('Temperature T')
plt.ylabel('Lambda Value')
plt.xscale('log')
plt.xlim(10**4.16, 2e8)
#plt.yscale('log')
plt.title('Lambda as a function of T')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Define temperature and radius grids
T_vals = np.logspace(4.16, 8.3, 500)      # Temperature range [K]
R_vals = np.logspace(0.14, 3.38, 500) * ktc   # Radius range [code units to physical, adjust as needed]

# Create meshgrid for evaluation
T_mesh, R_mesh = np.meshgrid(T_vals, R_vals, indexing='ij')

# Evaluate Lambda for each (T, R) pair
Lambda_grid = np.zeros_like(T_mesh)

for i in range(T_mesh.shape[0]):
    for j in range(T_mesh.shape[1]):
        T = T_mesh[i, j]
        R = R_mesh[i, j]
        Lambda_grid[i, j] = lf.Lambdacalc(T, R, "a", 1)  # Replace "a", 1 with actual args if needed

# Convert to log10 for plotting clarity
log_Lambda = np.log10(Lambda_grid)

# Plotting
plt.figure(figsize=(8, 6))
pcm = plt.pcolormesh(R_vals / ktc, T_vals, log_Lambda, shading='auto', cmap='viridis')
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Radius [in units of ktc]")
plt.ylabel("Temperature [K]")
plt.title("log10(Lambda) as function of Radius and Temperature")
cbar = plt.colorbar(pcm, label=r"$\log_{10}(\Lambda)$")
plt.grid(True, which='both', ls='--', lw=0.5)
plt.tight_layout()
plt.show()